## Example Binary Classification using proposed AutoML API
### Disclaimer: This is for demo and private preview purposes only.
### Disclaimer: This notebook uses features that may be broken or unsupported at any time.

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"
#i "nuget:https://mlnetcli.blob.core.windows.net/mlnetcli/index.json"

Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json

In [1]:
#r "nuget:MLNetAutoML.InteractiveExtension,0.1.1"
#r "nuget:XPlot.Plotly.Interactive,4.0.6"
#r "nuget:Microsoft.ML.AutoML,0.20.5-dev.22179.1"
#r "nuget:Microsoft.Data.Analysis,0.20.5-dev.22179.1"

Restore sources https://mlnetcli.blob.core.windows.net/mlnetcli/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.5-dev.22179.1 Microsoft.ML.AutoML, 0.20.5-dev.22179.1 MLNetAutoML.InteractiveExtension, 0.1.1 XPlot.Plotly.Interactive, 4.0.6

In [1]:

// Import usings.
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using MLNetAutoML.InteractiveExtension;


In [1]:
//Load File
var trainDataPath = Path.Combine(Directory.GetCurrentDirectory(), "titanic.csv");
var df = DataFrame.LoadCsv(trainDataPath);

//Setup AutoML Pipeline
var context = new MLContext();

var pipeline = context.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"Sex", @"Sex"), new InputOutputColumnPair(@"Embarked", @"Embarked") })
                           .Append(context.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"Pclass", @"Pclass"), new InputOutputColumnPair(@"Age", @"Age"), new InputOutputColumnPair(@"SibSp", @"SibSp"), new InputOutputColumnPair(@"Parch", @"Parch"), new InputOutputColumnPair(@"Fare", @"Fare") }))
                           .Append(context.Transforms.Concatenate(@"TextFeature", @"Name", "Ticket", "Cabin"))
                           .Append(context.Transforms.Text.FeaturizeText("TextFeature", "TextFeature"))
                           .Append(context.Transforms.Concatenate(@"Features", new[] { @"Sex", @"Embarked", @"Pclass", @"Age", @"SibSp", @"Parch", @"Fare", "TextFeature" }))
                           .Append(context.Transforms.Conversion.ConvertType("Survived", "Survived", DataKind.Boolean))
						   .Append(context.Auto().BinaryClassification(labelColumnName: "Survived", useFastForest: false));


// Configure AutoML
var trainTestSplit = context.Data.TrainTestSplit(df, 0.1);
var monitor = new NotebookMonitor();

var experiment = context.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(50)
                    .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
                    .SetEvaluateMetric(BinaryClassificationMetric.Accuracy, "Survived", "PredictedLabel")
                    .SetMonitor(monitor);

// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

// Start Experiment
var res = experiment.Run().Result;

index,Trial,Metric,Pipeline
0,0,0.67948717,Unknown=>SdcaLogisticRegressionBinary
1,1,0.7692308,Unknown=>SdcaLogisticRegressionBinary
2,2,0.82051283,Unknown=>LbfgsLogisticRegressionBinary
3,3,0.82051283,Unknown=>LightGbmBinary
4,4,0.82051283,Unknown=>FastTreeBinary
5,5,0.84615386,Unknown=>LightGbmBinary
6,6,0.82051283,Unknown=>LbfgsLogisticRegressionBinary
7,7,0.7051282,Unknown=>SdcaLogisticRegressionBinary
8,8,0.8076923,Unknown=>LightGbmBinary
9,9,0.6923077,Unknown=>SdcaLogisticRegressionBinary
